<a href="https://colab.research.google.com/github/carlos-alves-one/-ML-Zoomcamp-Week-6/blob/main/ML_Zoomcamp_Week_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Goldsmiths University of London
**Author....: Carlos Manuel de Oliveira Alves**<br>
**Student..: cdeol003**<br>
**Created..: 10/10/2022**

In [158]:
# Import all necessary libraries that we will use in this project
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

In [159]:
# Download the dataset for the project
data = data = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-06-trees/CreditScoring.csv'

!wget $data

--2022-10-13 10:26:49--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-06-trees/CreditScoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 182489 (178K) [text/plain]
Saving to: ‘CreditScoring.csv.5’

CreditScoring.csv.5 100%[===================>] 178.21K  --.-KB/s    in 0.03s   

2022-10-13 10:26:49 (6.07 MB/s) - ‘CreditScoring.csv.5’ saved [182489/182489]



In [160]:
# Print the column names and the first 9 records of the dataset
!head CreditScoring.csv

"Status","Seniority","Home","Time","Age","Marital","Records","Job","Expenses","Income","Assets","Debt","Amount","Price"
1,9,1,60,30,2,1,3,73,129,0,0,800,846
1,17,1,60,58,3,1,1,48,131,0,0,1000,1658
2,10,2,36,46,2,2,3,90,200,3000,0,2000,2985
1,0,1,60,24,1,1,1,63,182,2500,0,900,1325
1,0,1,36,26,1,1,1,46,107,0,0,310,910
1,1,2,60,36,2,1,1,75,214,3500,0,650,1645
1,29,2,60,44,2,1,1,75,125,10000,0,1600,1800
1,9,5,12,27,1,1,1,35,80,0,0,200,1093
1,0,2,60,32,2,1,3,90,107,15000,0,1200,1957


In [161]:
# Print the data now using Pandas
df = pd.read_csv(data)

In [162]:
# Print the first 5 rows of our daraset
df.head()

,Status,Seniority,Home,Time,Age,Marital,Records,Job,Expenses,Income,Assets,Debt,Amount,Price
0,1,9,1,60,30,2,1,3,73,129,0,0,800,846
1,1,17,1,60,58,3,1,1,48,131,0,0,1000,1658
2,2,10,2,36,46,2,2,3,90,200,3000,0,2000,2985
3,1,0,1,60,24,1,1,1,63,182,2500,0,900,1325
4,1,0,1,36,26,1,1,1,46,107,0,0,310,910


In [163]:
# Preparing the data:

# Use the column status we want to translate the numbers into strings
# with text is easier to understand, what this status means

In [164]:
# Lower case all the column names of the dataset
df.columns = df.columns.str.lower()
df.head()

,status,seniority,home,time,age,marital,records,job,expenses,income,assets,debt,amount,price
0,1,9,1,60,30,2,1,3,73,129,0,0,800,846
1,1,17,1,60,58,3,1,1,48,131,0,0,1000,1658
2,2,10,2,36,46,2,2,3,90,200,3000,0,2000,2985
3,1,0,1,60,24,1,1,1,63,182,2500,0,900,1325
4,1,0,1,36,26,1,1,1,46,107,0,0,310,910


In [165]:
# Check the column status and how many values we have
df.status.value_counts()
# we can see we have a variable zero

1    3200
2    1254
0       1
Name: status, dtype: int64

In [166]:
# Convert the column status to string using the map function
# this function map converts to a dictionary every value from 
# the original data frame to some new value
df.status.map({1: 'ok', 2: 'default', 0: 'unk'}) # unk means unknown

0            ok
1            ok
2       default
3            ok
4            ok
         ...   
4450    default
4451         ok
4452    default
4453         ok
4454         ok
Name: status, Length: 4455, dtype: object

In [167]:
# Check our dataframe after used the map function
df.head()

,status,seniority,home,time,age,marital,records,job,expenses,income,assets,debt,amount,price
0,1,9,1,60,30,2,1,3,73,129,0,0,800,846
1,1,17,1,60,58,3,1,1,48,131,0,0,1000,1658
2,2,10,2,36,46,2,2,3,90,200,3000,0,2000,2985
3,1,0,1,60,24,1,1,1,63,182,2500,0,900,1325
4,1,0,1,36,26,1,1,1,46,107,0,0,310,910


In [168]:
# Now dealing with values: home, marital, records and job
home_values = {
    1: 'rent',
    2: 'owner',
    3: 'private',
    4: 'ignore',
    5: 'parents',
    6: 'other',
    0: 'unk'
}

df.home = df.home.map(home_values)

marital_values = {
    1: 'single',
    2: 'married',
    3: 'widow',
    4: 'separated',
    5: 'divorced',
    0: 'unk'
}

df.marital = df.marital.map(marital_values)

records_values = {
    1: 'no',
    2: 'yes',
    0: 'unk'
}

df.records = df.records.map(records_values)

job_values = {
    1: 'fixed',
    2: 'partime',
    3: 'freelance',
    4: 'others',
    0: 'unk'
}

df.job = df.job.map(job_values)

In [169]:
# Check our data frame after used the map function
df.head()

,status,seniority,home,time,age,marital,records,job,expenses,income,assets,debt,amount,price
0,1,9,rent,60,30,married,no,freelance,73,129,0,0,800,846
1,1,17,rent,60,58,widow,no,fixed,48,131,0,0,1000,1658
2,2,10,owner,36,46,married,yes,freelance,90,200,3000,0,2000,2985
3,1,0,rent,60,24,single,no,fixed,63,182,2500,0,900,1325
4,1,0,rent,36,26,single,no,fixed,46,107,0,0,310,910


In [170]:
# Use the method describe that describes the dataset like all the statistics
df.describe()

,status,seniority,time,age,expenses,income,assets,debt,amount,price
count,4455.000000,4455.000000,4455.000000,4455.000000,4455.000000,4.455000e+03,4.455000e+03,4.455000e+03,4455.000000,4455.000000
mean,1.281257,7.987205,46.441751,37.077666,55.568799,7.633170e+05,1.060341e+06,4.043820e+05,1039.021773,1462.875645
std,0.450162,8.173444,14.655225,10.984856,19.515878,8.703625e+06,1.021757e+07,6.344253e+06,474.543007,628.089913
min,0.000000,0.000000,6.000000,18.000000,35.000000,0.000000e+00,0.000000e+00,0.000000e+00,100.000000,105.000000
25%,1.000000,2.000000,36.000000,28.000000,35.000000,8.000000e+01,0.000000e+00,0.000000e+00,700.000000,1117.500000
50%,1.000000,5.000000,48.000000,36.000000,51.000000,1.200000e+02,3.500000e+03,0.000000e+00,1000.000000,1400.000000
75%,2.000000,12.000000,60.000000,45.000000,72.000000,1.660000e+02,6.000000e+03,0.000000e+00,1300.000000,1692.000000
max,2.000000,48.000000,72.000000,68.000000,180.000000,1.000000e+08,1.000000e+08,1.000000e+08,5000.000000,11140.000000


In [171]:
# Use the method describe with the random function
df.describe().round()
# below we can see income, assets and debt have large numbers as max values
# this max values means the value is missing

,status,seniority,time,age,expenses,income,assets,debt,amount,price
count,4455.0,4455.0,4455.0,4455.0,4455.0,4455.0,4455.0,4455.0,4455.0,4455.0
mean,1.0,8.0,46.0,37.0,56.0,763317.0,1060341.0,404382.0,1039.0,1463.0
std,0.0,8.0,15.0,11.0,20.0,8703625.0,10217569.0,6344253.0,475.0,628.0
min,0.0,0.0,6.0,18.0,35.0,0.0,0.0,0.0,100.0,105.0
25%,1.0,2.0,36.0,28.0,35.0,80.0,0.0,0.0,700.0,1118.0
50%,1.0,5.0,48.0,36.0,51.0,120.0,3500.0,0.0,1000.0,1400.0
75%,2.0,12.0,60.0,45.0,72.0,166.0,6000.0,0.0,1300.0,1692.0
max,2.0,48.0,72.0,68.0,180.0,99999999.0,99999999.0,99999999.0,5000.0,11140.0


In [172]:
# Using another way similar to map function to replace values
df.income.replace(to_replace=99999999, value=np.nan)

0       129.0
1       131.0
2       200.0
3       182.0
4       107.0
        ...  
4450     92.0
4451     75.0
4452     90.0
4453    140.0
4454    140.0
Name: income, Length: 4455, dtype: float64

In [173]:
# Create a for loop to update the max value with NaN
for c in ['income', 'assets','debt']:

  # After access the column c we want to replace the value with NaN
  df [c] = df[c].replace(to_replace=99999999, value=np.nan)

In [174]:
# Check the max value is gone from the dataframe
df.describe().round() 
# we can see below we don't have the max value anymore

,status,seniority,time,age,expenses,income,assets,debt,amount,price
count,4455.0,4455.0,4455.0,4455.0,4455.0,4421.0,4408.0,4437.0,4455.0,4455.0
mean,1.0,8.0,46.0,37.0,56.0,131.0,5403.0,343.0,1039.0,1463.0
std,0.0,8.0,15.0,11.0,20.0,86.0,11573.0,1246.0,475.0,628.0
min,0.0,0.0,6.0,18.0,35.0,0.0,0.0,0.0,100.0,105.0
25%,1.0,2.0,36.0,28.0,35.0,80.0,0.0,0.0,700.0,1118.0
50%,1.0,5.0,48.0,36.0,51.0,120.0,3000.0,0.0,1000.0,1400.0
75%,2.0,12.0,60.0,45.0,72.0,165.0,6000.0,0.0,1300.0,1692.0
max,2.0,48.0,72.0,68.0,180.0,959.0,300000.0,30000.0,5000.0,11140.0


In [175]:
# Look at the status variable. We have zeros meaning we have missing values
df.status.value_counts()
# below the status zero means unk and we want to remove it

1    3200
2    1254
0       1
Name: status, dtype: int64

In [176]:
# Filter all records with status, not zero, from the data frame
df[df.status != 0]

,status,seniority,home,time,age,marital,records,job,expenses,income,assets,debt,amount,price
0,1,9,rent,60,30,married,no,freelance,73,129.0,0.0,0.0,800,846
1,1,17,rent,60,58,widow,no,fixed,48,131.0,0.0,0.0,1000,1658
2,2,10,owner,36,46,married,yes,freelance,90,200.0,3000.0,0.0,2000,2985
3,1,0,rent,60,24,single,no,fixed,63,182.0,2500.0,0.0,900,1325
4,1,0,rent,36,26,single,no,fixed,46,107.0,0.0,0.0,310,910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4450,2,1,rent,60,39,married,no,fixed,69,92.0,0.0,0.0,900,1020
4451,1,22,owner,60,46,married,no,fixed,60,75.0,3000.0,600.0,950,1263
4452,2,0,owner,24,37,married,no,partime,60,90.0,3500.0,0.0,500,963
4453,1,0,rent,48,23,single,no,freelance,49,140.0,0.0,0.0,550,550


In [177]:
# Reset the index of the data frame
df = df[df.status != 0].reset_index()
df

,index,status,seniority,home,time,age,marital,records,job,expenses,income,assets,debt,amount,price
0,0,1,9,rent,60,30,married,no,freelance,73,129.0,0.0,0.0,800,846
1,1,1,17,rent,60,58,widow,no,fixed,48,131.0,0.0,0.0,1000,1658
2,2,2,10,owner,36,46,married,yes,freelance,90,200.0,3000.0,0.0,2000,2985
3,3,1,0,rent,60,24,single,no,fixed,63,182.0,2500.0,0.0,900,1325
4,4,1,0,rent,36,26,single,no,fixed,46,107.0,0.0,0.0,310,910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4449,4450,2,1,rent,60,39,married,no,fixed,69,92.0,0.0,0.0,900,1020
4450,4451,1,22,owner,60,46,married,no,fixed,60,75.0,3000.0,600.0,950,1263
4451,4452,2,0,owner,24,37,married,no,partime,60,90.0,3500.0,0.0,500,963
4452,4453,1,0,rent,48,23,single,no,freelance,49,140.0,0.0,0.0,550,550


In [178]:
# Doing the train, validation and test split:

# Import library sklearn the package train test split
from sklearn.model_selection import train_test_split

# Split the data into full train and test dataframe
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=11)

# Split the data into train and validation dataframe
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=11)

In [179]:
# Reset the index of the train, validation and test dataframes
df_train = df_train.reset_index(drop=True)
df_val   = df_val.reset_index(drop=True)
df_test  = df_test.reset_index(drop=True)

In [180]:
# Turn the status to a number 1 = true and 0 = false and store in y_train variable
y_train = (df_train.status == 2).astype('int').values
y_train
# Do the same to validation and test dataframes
y_val  = (df_val.status == 2).astype('int').values
y_test = (df_test.status == 2).astype('int').values

In [181]:
# Remove the default variables from train, validation and test dataframes
del df_train['status']
del df_val['status']
del df_test['status']

In [182]:
df_train

,index,seniority,home,time,age,marital,records,job,expenses,income,assets,debt,amount,price
0,951,10,owner,36,36,married,no,freelance,75,0.0,10000.0,0.0,1000,1400
1,688,6,parents,48,32,single,yes,fixed,35,85.0,0.0,0.0,1100,1330
2,2233,1,parents,48,40,married,no,fixed,75,121.0,0.0,0.0,1320,1600
3,3304,1,parents,48,23,single,no,partime,35,72.0,0.0,0.0,1078,1079
4,2271,5,owner,36,46,married,no,freelance,60,100.0,4000.0,0.0,1100,1897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2667,2382,18,private,36,45,married,no,fixed,45,220.0,20000.0,0.0,800,1600
2668,1784,7,private,60,29,married,no,fixed,60,51.0,3500.0,500.0,1000,1290
2669,808,1,parents,24,19,single,no,fixed,35,28.0,0.0,0.0,400,600
2670,1857,15,owner,48,43,married,no,freelance,60,100.0,18000.0,0.0,2500,2976


In [183]:
# Applying Decision Trees:

# Create asses risk function
def assess_risk(client):

  # Check the customer has a record
  if client['records'] == 'yes':

    # Check the customer job is part-time
    if client['job'] == 'parttime':
      return 'default'
    else:
      return 'ok'
  else:
    # Check the customer have more than 6k in assets
    if client['assets'] > 6000:
      return 'ok'
    else:
      return 'default'

In [184]:
# Convert the first record from the train dataframe into a dictionary
xi = df_train.iloc[0].to_dict()
xi

{'index': 951,
 'seniority': 10,
 'home': 'owner',
 'time': 36,
 'age': 36,
 'marital': 'married',
 'records': 'no',
 'job': 'freelance',
 'expenses': 75,
 'income': 0.0,
 'assets': 10000.0,
 'debt': 0.0,
 'amount': 1000,
 'price': 1400}

In [185]:
# Use the assess risk function with our customer
assess_risk(xi)

'ok'

In [186]:
# Now we can train using the rules created above
# so we can learn these rules from the data using the decision tree algorithm
# and for that we are using the sklearn

# Import sklearn with the package Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier

In [187]:
# Use dictionary vectorizer because we have categorical variables
from sklearn.feature_extraction import DictVectorizer

In [188]:
# Import the library warnings to ignore the warnings
import warnings
warnings.filterwarnings("ignore")

In [189]:
# Turn our train dataframe into a list of dictionaries
# and then turn the list of dictionaries into the feature matrix
# and then after we will train the model
# Use the fillna function to fill the NaN with zeros
train_dicts = df_train.fillna(0).to_dict(orient='records')

In [190]:
# Create a new instance of the DictVectorizer class without sparse
dv = DictVectorizer(sparse=False)

# Use the function transform with our DictVectorizer
X_train = dv.fit_transform(train_dicts)

In [191]:
# Look and check at the feature names
dv.get_feature_names()

['age',
 'amount',
 'assets',
 'debt',
 'expenses',
 'home=ignore',
 'home=other',
 'home=owner',
 'home=parents',
 'home=private',
 'home=rent',
 'home=unk',
 'income',
 'index',
 'job=fixed',
 'job=freelance',
 'job=others',
 'job=partime',
 'job=unk',
 'marital=divorced',
 'marital=married',
 'marital=separated',
 'marital=single',
 'marital=unk',
 'marital=widow',
 'price',
 'records=no',
 'records=yes',
 'seniority',
 'time']

In [192]:
# Train our decision tree
dt = DecisionTreeClassifier()

# Use the function transform with our DictVectorizer
X_train = dv.fit_transform(train_dicts)